In [1]:
import sqlite3
import pandas as pd

### Подключение к БД

In [2]:
conn = sqlite3.connect("data.db")

# Необходимо уметь приводить все символы к нижнему регистру, sqlite умеет только англ
def sqlite_lower(value_):
    return str(value_).lower()


conn.create_function("LOWER", 1, sqlite_lower)
conn.commit()

Выбираем все договоры самой старшей версии с меткой emergency - "Закупка осуществляется вследствие аварии,
 иных чрезвычайных ситуаций природного или техногенного характера, непреодолимой силы,
 принеобходимости срочного медицинского вмешательства, а также для предотвращения угрозы
 возникновения указанных ситуаций"

In [3]:
covid_contracts = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    """,
    conn
)

covid_contracts

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,purchase_notice_info_number,lot_guid,subject_contract,purchase_type_code,contract_price,contract_rub_price,contract_currency_code,contract_start_execution_date,contract_end_execution_date,region_name
0,00015975-9c32-4b00-922d-7ab2866d3cf7,51433000147200015550001,1,0,0,0,None,None,2020-12-01 00:32:23,1433000147,...,None,None,None,30000,5.342400e+06,NaN,RUB,2020-10-15,2020-12-17,Saha_Jakutiya_Resp
1,0008f92e-3856-4383-97d2-2fd8b9b71aee,57708503727200082220007,1,0,0,0,None,None,2021-09-23 12:34:16,7708503727,...,None,None,Выполнение комплекса аварийно-восстановительны...,30000,1.016760e+09,1.016760e+09,RUB,2020-05-31,2021-12-30,Moskva
2,000dc3f6-e39c-456b-b321-5486e2d8b78e,76652022897210001260001,1,0,0,0,None,40451834-df9b-11eb-b80c-b8cef612661b,2021-07-23 04:46:32,6652022897,...,None,None,1719-2021-00259. Выполнение работ по ремонту э...,30000,1.194000e+05,1.194000e+05,RUB,2021-07-21,2021-08-20,Sverdlovskaya_obl
3,000efbce-29cc-4fda-ba27-e5b895f5b6ad,56684021776200003580001,0,1,0,0,None,None,2020-12-02 05:02:53,6684021776,...,32009740825,0621d71c-65f9-4181-81d5-b9e077fab7ef,Поставка лекарственных препаратов,40000,2.453075e+06,2.453075e+06,RUB,2020-11-29,2021-02-27,Sverdlovskaya_obl
4,000f310c-5934-4d63-be2e-74dd4c99e301,56611000527200002950001,1,0,0,0,None,ed850d8e-b314-4c08-92b1-e49429220e75,2020-10-27 06:07:26,6611000527,...,None,None,Поставка СИЗ: перчатки,30000,2.408000e+05,2.408000e+05,RUB,2020-10-25,2020-12-30,Sverdlovskaya_obl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16638,fff92ffe-8c3f-443f-8ca1-54b8cb98bb64,53801013663210002050001,1,0,0,0,None,None,2021-10-04 02:31:36,3801013663,...,None,None,Поставка лекарственного препарата (гепарин),30000,1.341640e+05,NaN,RUB,2021-10-03,2021-10-30,Irkutskaya_obl
16639,fffa65c3-f306-4bf8-a55c-89d740483fc5,57702070139200015180001,1,0,0,0,None,None,2020-06-04 06:38:35,7702070139,...,None,None,Поставка масок одноразовых,30000,2.800000e+06,NaN,RUB,2020-05-18,2020-06-03,Sankt-Peterburg
16640,fffaff02-11f4-4936-9d2c-01cc2b4c1c8d,58601053210200000270001,1,0,0,0,None,None,2020-10-02 05:04:25,8601053210,...,None,None,неисключительное право на воспроизведение прог...,30000,1.250372e+05,NaN,RUB,2020-10-01,2020-12-30,Hanty-Mansiiskii_AO_Iugra_AO
16641,fffb9623-244a-4c27-b1c7-4518ab6412bd,51431004812210000980001,0,0,0,0,None,None,2021-07-07 09:05:27,1431004812,...,32110451253,89d200ae-e338-463b-89b3-0cf5955eee3c,Поставка лекарственных средств (артлегиа),40000,6.006000e+05,NaN,RUB,2021-07-06,2021-12-30,Saha_Jakutiya_Resp


Для дальнейшего анализа необходимо проверить наличие поля с указанием лота

In [4]:
covid_contracts_with_not_null_lot = covid_contracts[covid_contracts["lot_guid"].notnull()]

print(f'Количество "ковидных" договоров: {covid_contracts.shape[0]}')
print(f'Количество "ковидных" договоров с указанием лота: {covid_contracts_with_not_null_lot.shape[0]}')

Количество "ковидных" договоров: 16643
Количество "ковидных" договоров с указанием лота: 7699


Договоры без указания lot_guid, это те где извещение не размещалось в соответствии
с частью 5 статьи 4 Федерального Закона № 223-ФЗ, например https://zakupki.gov.ru/epz/contractfz223/card/contract-info.html?id=9268727

Для выборки будем использовать только те договоры, в которых присутствуют лоты

К сожалению, полученная выборка слишком мала. Попробуем расширить её, использую предмет договора

In [5]:
COVID_CONDITIONS = """ (
emergency = 1
or lower(subject_contract) like '%sars%cov%'
or lower(subject_contract) like '%covid%'
or lower(subject_contract) like '%коронавирус%'
or lower(subject_contract) like '%противовирусн%'
or lower(subject_contract) like '%пцр%'
or lower(subject_contract) like '%пандеми%'
or lower(subject_contract) like '%маск%медицинск%'
or lower(subject_contract) like '%медицинск%маск%'
or lower(subject_contract) like '%медицинск%масок%'
or lower(subject_contract) like '%дезинфекц%'
or lower(subject_contract) like '%дезинфиц%'
or lower(subject_contract) like '%аппарат%ивл%'
or lower(subject_contract) like '%искусственн%вентиляц%л_гк%'
or lower(subject_contract) like '%масок%медицинск%'
or lower(subject_contract) like '%одноразов%масок%'
or lower(subject_contract) like '%одноразов%маск%'
or lower(subject_contract) like '%маск%одноразов%'
or lower(subject_contract) like '%масок%одноразов%'
or lower(subject_contract) like '%антисепти%'
or lower(subject_contract) like '%термометр%'
or lower(subject_contract) like '%контрол%температур%'
or lower(subject_contract) like '%медицинск%перчат%'
or lower(subject_contract) like '%перчат%медицинск%'
or lower(subject_contract) like '%медиц%кислород%'
or lower(subject_contract) like '%кислород%медиц%'
)
"""
covid_contracts = pd.read_sql_query(
    f"""
    SELECT COUNT(*)
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE {COVID_CONDITIONS}
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    """,
    conn
)

covid_contracts

,COUNT(*)
0,32175


Проверим, существуют ли лоты, на которые было заключено несколько договоров.

In [6]:
lots_with_many_contracts = pd.read_sql_query(
    f"""
    SELECT * FROM (
        SELECT lot_guid, COUNT(*) as c
            FROM contract c
            INNER JOIN (
                SELECT contract_reg_number, MAX(version) ver
                FROM contract
                WHERE {COVID_CONDITIONS}
                and lot_guid is not null
                GROUP BY contract_reg_number
            ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
        GROUP BY lot_guid
    ) WHERE c > 1
    """,
    conn
)

lots_with_many_contracts

,lot_guid,c
0,0073f630-28e6-41af-ab51-5d2996956007,2
1,03fd4534-23ea-4cf7-9591-069f6ac23de1,13
2,04e89b57-23f6-4c2d-a7f9-aa74884104e5,2
3,067223a9-c129-4110-865a-8a0f0fa283cc,2
4,06f56a5d-372f-b19f-87da-caec33c8af02,7
...,...,...
353,fdff2801-bed2-4ed1-83b5-1b82b0ea93c5,2
354,fe96d4a7-3c6f-43d1-b937-4e4ac68e19ea,2
355,fee77238-6233-44dc-83a4-a94f2471a6b0,17
356,ff1be10a-721c-4075-b5e1-13ae00f5f001,4


Такие договоры существуют.  Необходимо будет добавить информацию об этом в выборку.

Выберем из базы договоры с лотами

In [7]:
covid_contracts_with_lot = pd.read_sql_query(
    f"""
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name,
    pnld.initial_sum as notice_lot_initial_sum, pnld.starting_contract_price_rub as notice_lot_starting_contract_price_rub,
    pnld.currency_code as notice_lot_currency_code,
    pnld.commodity_price as notice_lot_commodity_price, pnld.commodity_price_rub as notice_lot_commodity_price_rub,
    pnld.max_contract_price as notice_lot_max_contract_price, pnld.max_contract_price_rub as notice_lot_max_contract_price_rub
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE {COVID_CONDITIONS}
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    """,
    conn
)

covid_contracts_with_lot

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,contract_start_execution_date,contract_end_execution_date,region_name,notice_lot_initial_sum,notice_lot_starting_contract_price_rub,notice_lot_currency_code,notice_lot_commodity_price,notice_lot_commodity_price_rub,notice_lot_max_contract_price,notice_lot_max_contract_price_rub
0,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,2021-01-11,2021-06-29,Tomskaya_obl,550800.00,NaN,RUB,None,None,NaN,NaN
1,0004ff4b-67ff-42b6-a163-88e556affbeb,50274946336210000830007,0,0,0,0,None,None,2021-11-29 10:29:58,0274946336,...,2021-04-19,2022-12-30,Bashkortostan_Resp,4415139.00,NaN,RUB,None,None,NaN,NaN
2,000ab495-ae5e-42c4-9643-94083ebb7ceb,59111006597210000080013,0,0,0,0,None,None,2021-10-27 09:33:09,9111006597,...,2021-08-09,2022-05-30,Krym_Resp,1998600.00,NaN,RUB,None,None,NaN,NaN
3,000ccb0b-8346-4f62-bb32-04d284ba6fca,56027024472210000150001,0,0,0,0,None,None,2021-06-21 10:50:00,6027024472,...,2021-06-20,2021-08-30,Pskovskaya_obl,1143188.00,NaN,RUB,None,None,NaN,NaN
4,000d73ea-90c4-460f-8074-2d8eb70aa733,83664056479200000350001,0,0,0,0,None,None,2020-06-29 05:37:38,3664056479,...,2020-06-28,2020-12-30,Voronezhskaya_obl,4974515.45,NaN,RUB,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30698,fff5ad7e-5884-46f1-9f07-4150e8cc6258,52801022793210003140001,0,0,0,0,None,None,2021-11-01 05:30:25,2801022793,...,2021-10-31,2021-11-09,Amurskaya_obl,365400.00,NaN,RUB,None,None,NaN,NaN
30699,fff6f724-be51-4180-930f-41999fdd5497,51435149216210002250001,0,0,0,0,None,None,2021-11-25 02:46:38,1435149216,...,2021-11-24,2021-12-30,Saha_Jakutiya_Resp,686400.00,NaN,RUB,None,None,NaN,NaN
30700,fff8c702-ba6a-48c2-90d4-67c2c857cfd1,67725038124200003980001,0,0,0,0,None,None,2020-11-20 08:42:35,7725038124,...,2020-11-16,None,Moskva,23705828.40,NaN,RUB,None,None,NaN,NaN
30701,fffb9623-244a-4c27-b1c7-4518ab6412bd,51431004812210000980001,0,0,0,0,None,None,2021-07-07 09:05:27,1431004812,...,2021-07-06,2021-12-30,Saha_Jakutiya_Resp,600600.00,NaN,RUB,None,None,NaN,NaN


Проверим, существуют ли договоры, у которых заказчик отличается от заказчика в извещении

In [8]:
contracts_with_different_customer = pd.read_sql_query(
    f"""
    SELECT c.guid, c.customer_inn, c.placer_inn, c.detached_org_inn, pn.guid, pn.customer_inn, pn.placer_inn, pn.detached_org_inn
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE {COVID_CONDITIONS}
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl on c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_to_lot pntl on pnl.guid = pntl.purchase_notice_lot_guid
    LEFT JOIN purchase_notice pn on pntl.purchase_notice_guid = pn.guid
    WHERE (c.customer_inn != pn.customer_inn or c.placer_inn != pn.placer_inn)
    ;
    """,
    conn
)

contracts_with_different_customer

,guid,customer_inn,placer_inn,detached_org_inn,guid,customer_inn,placer_inn,detached_org_inn
0,000d73ea-90c4-460f-8074-2d8eb70aa733,3664056479,3664056479,None,1cb41fb5-1c6e-47e4-a123-13a39cf27d46,3664056479,3662158965,None
1,0018a028-28b4-4f82-a1a3-802bcf99dd89,7820309254,7820309254,None,696da96f-20ee-4e87-bcdd-2248ee08d63f,7708619320,7708619320,None
2,002307d1-8cba-4a17-a125-104668f2558f,1654038766,1654038766,1654038766,949dedf2-ff3a-401e-abbc-b5c9bca976b8,1654038766,1660291100,None
3,003bbfbc-93d1-49e2-8555-e8447bc9e99c,2902037066,2902037066,2902037066,645ea858-2470-b86e-0b24-52e8a596865e,2902037066,2901252568,None
4,003f2613-8a83-482f-8546-b07df2a3ee86,3908010986,3908010986,None,fea8d150-1c42-466b-a5a9-3f3e5550ede0,3908010986,3904612235,None
...,...,...,...,...,...,...,...,...
4578,ffd95f92-f12b-4488-9a93-f0fd06c2c38e,1101481574,1101481574,1101481574,e877d410-a198-440e-a893-24b62e54745d,1101481574,7707308480,None
4579,ffea4edc-1b55-4df6-b429-f485f18cd350,7724490000,7724490000,None,2b7ea3c4-dd16-45f4-bbb5-737b1357a3fc,7724490000,7838430413,7724490000
4580,fff544f3-3733-4943-b6db-1a9662558747,7708245723,7708245723,7708245723,c39dbd31-9f6f-4d23-83b2-0cdc960a7985,7708245723,7707308480,None
4581,fff6f724-be51-4180-930f-41999fdd5497,1435149216,1435149216,None,1dea37d1-6ff1-4eed-8de4-f28d7ca17be2,1435149216,7710357167,None


Такие договоры существуют, поэтому необходимо будет добавить данные об изначальном заказчике в выборку

Дополним информацией о том, был ли договор завершен или расторгнут

In [9]:
covid_contracts_performance_with_lot = pd.read_sql_query(
    f"""
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name,
    pnld.initial_sum as notice_lot_initial_sum, pnld.starting_contract_price_rub as notice_lot_starting_contract_price_rub,
    pnld.currency_code as notice_lot_currency_code,
    pnld.commodity_price as notice_lot_commodity_price, pnld.commodity_price_rub as notice_lot_commodity_price_rub,
    pnld.max_contract_price as notice_lot_max_contract_price, pnld.max_contract_price_rub as notice_lot_max_contract_price_rub,
    cp.is_complete as contract_is_complete
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE {COVID_CONDITIONS}
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    LEFT JOIN (
        SELECT c2.contract_reg_number, MIN(is_complete) as is_complete
        FROM contract c2
        LEFT JOIN contract_performance cp2 ON c2.contract_reg_number = cp2.contract_registration_number
        AND c2.lot_guid is not null
        GROUP BY c2.contract_reg_number
    ) cp ON cp.contract_reg_number = c.contract_reg_number
    """,
    conn
)

covid_contracts_performance_with_lot

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,contract_end_execution_date,region_name,notice_lot_initial_sum,notice_lot_starting_contract_price_rub,notice_lot_currency_code,notice_lot_commodity_price,notice_lot_commodity_price_rub,notice_lot_max_contract_price,notice_lot_max_contract_price_rub,contract_is_complete
0,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,2021-06-29,Tomskaya_obl,550800.00,NaN,RUB,None,None,NaN,NaN,1.0
1,0004ff4b-67ff-42b6-a163-88e556affbeb,50274946336210000830007,0,0,0,0,None,None,2021-11-29 10:29:58,0274946336,...,2022-12-30,Bashkortostan_Resp,4415139.00,NaN,RUB,None,None,NaN,NaN,1.0
2,000ab495-ae5e-42c4-9643-94083ebb7ceb,59111006597210000080013,0,0,0,0,None,None,2021-10-27 09:33:09,9111006597,...,2022-05-30,Krym_Resp,1998600.00,NaN,RUB,None,None,NaN,NaN,1.0
3,000ccb0b-8346-4f62-bb32-04d284ba6fca,56027024472210000150001,0,0,0,0,None,None,2021-06-21 10:50:00,6027024472,...,2021-08-30,Pskovskaya_obl,1143188.00,NaN,RUB,None,None,NaN,NaN,1.0
4,000d73ea-90c4-460f-8074-2d8eb70aa733,83664056479200000350001,0,0,0,0,None,None,2020-06-29 05:37:38,3664056479,...,2020-12-30,Voronezhskaya_obl,4974515.45,NaN,RUB,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30698,fff5ad7e-5884-46f1-9f07-4150e8cc6258,52801022793210003140001,0,0,0,0,None,None,2021-11-01 05:30:25,2801022793,...,2021-11-09,Amurskaya_obl,365400.00,NaN,RUB,None,None,NaN,NaN,NaN
30699,fff6f724-be51-4180-930f-41999fdd5497,51435149216210002250001,0,0,0,0,None,None,2021-11-25 02:46:38,1435149216,...,2021-12-30,Saha_Jakutiya_Resp,686400.00,NaN,RUB,None,None,NaN,NaN,NaN
30700,fff8c702-ba6a-48c2-90d4-67c2c857cfd1,67725038124200003980001,0,0,0,0,None,None,2020-11-20 08:42:35,7725038124,...,None,Moskva,23705828.40,NaN,RUB,None,None,NaN,NaN,0.0
30701,fffb9623-244a-4c27-b1c7-4518ab6412bd,51431004812210000980001,0,0,0,0,None,None,2021-07-07 09:05:27,1431004812,...,2021-12-30,Saha_Jakutiya_Resp,600600.00,NaN,RUB,None,None,NaN,NaN,1.0


Необходимо дополнить выборку информацией о позициях договора

In [10]:
covid_contracts_positions_with_lot = pd.read_sql_query(
    f"""
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name,
    pnld.initial_sum as notice_lot_initial_sum, pnld.starting_contract_price_rub as notice_lot_starting_contract_price_rub,
    pnld.currency_code as notice_lot_currency_code,
    pnld.commodity_price as notice_lot_commodity_price, pnld.commodity_price_rub as notice_lot_commodity_price_rub,
    pnld.max_contract_price as notice_lot_max_contract_price, pnld.max_contract_price_rub as notice_lot_max_contract_price_rub,
    cp.is_complete as contract_is_complete,
    cpos.name as contract_position_name,
    cpos.okpd2_code as position_okpd2_code, cpos.okpd2_name as position_okpd2_name,
    cpos.country_code as position_country_code, cpos.producer_country as position_producer_country,
    cpos.impossible_to_determine_attr as position_impossible_to_determine_attr, cpos.okei_code as position_okei_code,
    cpos.okei_name as position_okei_name, cpos.qty as position_qty, cpos.unit_price as position_unit_price,
    cpos.rub_unit_price as position_rub_unit_price, cpos.currency_code as position_currency_code
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE {COVID_CONDITIONS}
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    LEFT JOIN (
        SELECT c2.contract_reg_number, MIN(is_complete) as is_complete
        FROM contract c2
        LEFT JOIN contract_performance cp2 ON c2.contract_reg_number = cp2.contract_registration_number
        AND c2.lot_guid is not null
        GROUP BY c2.contract_reg_number
    ) cp ON cp.contract_reg_number = c.contract_reg_number
    LEFT JOIN position_to_contract ptc on c.guid = ptc.contract_guid
    LEFT JOIN contract_position cpos on ptc.position_guid = cpos.guid
    """,
    conn
)

covid_contracts_positions_with_lot

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_okpd2_name,position_country_code,position_producer_country,position_impossible_to_determine_attr,position_okei_code,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code
0,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Перчатки хирургические резиновые,None,None,0,715,Пара (2 шт.),4000.0,26.0,NaN,RUB
1,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Перчатки хирургические резиновые,None,None,0,715,Пара (2 шт.),5000.0,26.0,NaN,RUB
2,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Перчатки хирургические резиновые,None,None,0,715,Пара (2 шт.),1000.0,28.0,NaN,RUB
3,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Перчатки хирургические резиновые,None,None,0,715,Пара (2 шт.),100.0,54.0,NaN,RUB
4,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Перчатки хирургические резиновые,None,None,0,715,Пара (2 шт.),8000.0,26.0,NaN,RUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72190,fff5ad7e-5884-46f1-9f07-4150e8cc6258,52801022793210003140001,0,0,0,0,None,None,2021-11-01 05:30:25,2801022793,...,"Изделия медицинские, в том числе хирургические...",None,None,0,778,Упаковка,63.0,5800.0,NaN,RUB
72191,fff6f724-be51-4180-930f-41999fdd5497,51435149216210002250001,0,0,0,0,None,None,2021-11-25 02:46:38,1435149216,...,Препараты лекарственные,None,None,1,None,None,NaN,617760.0,NaN,RUB
72192,fff8c702-ba6a-48c2-90d4-67c2c857cfd1,67725038124200003980001,0,0,0,0,None,None,2020-11-20 08:42:35,7725038124,...,"Услуги по дезинфекции, дезинсекции и дератизации",None,None,0,876,Условная единица,1.0,NaN,NaN,None
72193,fffb9623-244a-4c27-b1c7-4518ab6412bd,51431004812210000980001,0,0,0,0,None,None,2021-07-07 09:05:27,1431004812,...,Препараты лекарственные,None,None,0,778,Упаковка,14.0,54600.0,NaN,RUB


Добавим информацию о том, сколько договоров заключено по лоту в выборку.

In [11]:
contracts_count_for_lot = pd.read_sql_query(
    f"""
    SELECT lot_guid, COUNT(*) as contracts_for_lot
        FROM contract c
        INNER JOIN (
            SELECT contract_reg_number, MAX(version) ver
            FROM contract
            WHERE {COVID_CONDITIONS}
            AND lot_guid is not null
            GROUP BY contract_reg_number
        ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    GROUP BY lot_guid
    """,
    conn
)

contracts_count_for_lot

,lot_guid,contracts_for_lot
0,0001eb03-1d53-4a0d-afed-d15afacc9c03,1
1,00063eeb-9fc5-4bd1-b269-9ebf61448d5d,1
2,00069375-47f6-40d8-8b20-df78140b1416,1
3,000b0a3c-7291-4077-836d-73b1ce173a73,1
4,000f785b-cde8-42fc-bdd4-dd401bf48e52,1
...,...,...
30677,fff43019-a748-4955-af26-6f4f1488571f,1
30678,fff43d4a-a42c-4665-bfc5-c50c164bf48b,1
30679,fff60f78-fc33-4858-ad9e-90605a125069,1
30680,fffdce4d-a139-4939-ada5-5bea3fb1d25e,1


In [12]:
dataset = pd.merge(
    covid_contracts_positions_with_lot,
    contracts_count_for_lot,
    how='left',
    on=['lot_guid']
)

dataset

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_country_code,position_producer_country,position_impossible_to_determine_attr,position_okei_code,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code,contracts_for_lot
0,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,None,None,0,715,Пара (2 шт.),4000.0,26.0,NaN,RUB,1
1,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,None,None,0,715,Пара (2 шт.),5000.0,26.0,NaN,RUB,1
2,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,None,None,0,715,Пара (2 шт.),1000.0,28.0,NaN,RUB,1
3,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,None,None,0,715,Пара (2 шт.),100.0,54.0,NaN,RUB,1
4,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,None,None,0,715,Пара (2 шт.),8000.0,26.0,NaN,RUB,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72190,fff5ad7e-5884-46f1-9f07-4150e8cc6258,52801022793210003140001,0,0,0,0,None,None,2021-11-01 05:30:25,2801022793,...,None,None,0,778,Упаковка,63.0,5800.0,NaN,RUB,1
72191,fff6f724-be51-4180-930f-41999fdd5497,51435149216210002250001,0,0,0,0,None,None,2021-11-25 02:46:38,1435149216,...,None,None,1,None,None,NaN,617760.0,NaN,RUB,1
72192,fff8c702-ba6a-48c2-90d4-67c2c857cfd1,67725038124200003980001,0,0,0,0,None,None,2020-11-20 08:42:35,7725038124,...,None,None,0,876,Условная единица,1.0,NaN,NaN,None,1
72193,fffb9623-244a-4c27-b1c7-4518ab6412bd,51431004812210000980001,0,0,0,0,None,None,2021-07-07 09:05:27,1431004812,...,None,None,0,778,Упаковка,14.0,54600.0,NaN,RUB,1


Добавим информацию об изначальных заказчиках в выборку

In [13]:
notice_customers_for_contract = pd.read_sql_query(
    f"""
    SELECT c.guid as contract_guid, pn.customer_inn as notice_customer_inn,
        pn.placer_inn as notice_placer_inn, pn.detached_org_inn as notice_detached_org_inn
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE {COVID_CONDITIONS}
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN (
        --если несколько извещений с одинаковым номером, выбераем последнее
        SELECT c2.guid, c2.purchase_notice_info_number, MAX(pn2.create_date_time) as max_date
        FROM contract c2
        INNER JOIN purchase_notice pn2 ON pn2.registration_number = c2.purchase_notice_info_number
        AND c2.purchase_notice_info_number is not null
        GROUP BY c2.guid, c2.purchase_notice_info_number
    ) tmp2 ON c.guid = tmp2.guid
    LEFT JOIN purchase_notice pn ON pn.create_date_time = tmp2.max_date
        AND pn.registration_number = tmp2.purchase_notice_info_number
    GROUP BY c.guid, pn.customer_inn, pn.placer_inn, pn.detached_org_inn
    """,
    conn
)

notice_customers_for_contract

,contract_guid,notice_customer_inn,notice_placer_inn,notice_detached_org_inn
0,00026a57-3035-45cd-9a0b-598d5b233a6b,7017000450,7017000450,None
1,0004ff4b-67ff-42b6-a163-88e556affbeb,0274946336,0274946336,None
2,000ab495-ae5e-42c4-9643-94083ebb7ceb,9111006597,9111006597,None
3,000ccb0b-8346-4f62-bb32-04d284ba6fca,6027024472,6027024472,None
4,000d73ea-90c4-460f-8074-2d8eb70aa733,3664056479,3662158965,None
...,...,...,...,...
30698,fff5ad7e-5884-46f1-9f07-4150e8cc6258,2801022793,2801022793,None
30699,fff6f724-be51-4180-930f-41999fdd5497,1435149216,7710357167,None
30700,fff8c702-ba6a-48c2-90d4-67c2c857cfd1,7725038124,7725038124,None
30701,fffb9623-244a-4c27-b1c7-4518ab6412bd,1431004812,1431004812,None


In [14]:
dataset = pd.merge(
    dataset,
    notice_customers_for_contract,
    how='left',
    on='contract_guid'
)

dataset

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_okei_code,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code,contracts_for_lot,notice_customer_inn,notice_placer_inn,notice_detached_org_inn
0,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,715,Пара (2 шт.),4000.0,26.0,NaN,RUB,1,7017000450,7017000450,None
1,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,715,Пара (2 шт.),5000.0,26.0,NaN,RUB,1,7017000450,7017000450,None
2,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,715,Пара (2 шт.),1000.0,28.0,NaN,RUB,1,7017000450,7017000450,None
3,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,715,Пара (2 шт.),100.0,54.0,NaN,RUB,1,7017000450,7017000450,None
4,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,715,Пара (2 шт.),8000.0,26.0,NaN,RUB,1,7017000450,7017000450,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72190,fff5ad7e-5884-46f1-9f07-4150e8cc6258,52801022793210003140001,0,0,0,0,None,None,2021-11-01 05:30:25,2801022793,...,778,Упаковка,63.0,5800.0,NaN,RUB,1,2801022793,2801022793,None
72191,fff6f724-be51-4180-930f-41999fdd5497,51435149216210002250001,0,0,0,0,None,None,2021-11-25 02:46:38,1435149216,...,None,None,NaN,617760.0,NaN,RUB,1,1435149216,7710357167,None
72192,fff8c702-ba6a-48c2-90d4-67c2c857cfd1,67725038124200003980001,0,0,0,0,None,None,2020-11-20 08:42:35,7725038124,...,876,Условная единица,1.0,NaN,NaN,None,1,7725038124,7725038124,None
72193,fffb9623-244a-4c27-b1c7-4518ab6412bd,51431004812210000980001,0,0,0,0,None,None,2021-07-07 09:05:27,1431004812,...,778,Упаковка,14.0,54600.0,NaN,RUB,1,1431004812,1431004812,None


Добавим информацию о том, сколько позиций в каждом контракте

In [15]:
contract_positions_count = pd.read_sql_query(
    f"""
    SELECT c.guid as contract_guid, COUNT(*) as contract_positions_count
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE {COVID_CONDITIONS}
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN position_to_contract ptc on c.guid = ptc.contract_guid
    LEFT JOIN contract_position cpos on ptc.position_guid = cpos.guid
    GROUP BY c.guid
    """,
    conn
)

contract_positions_count

,contract_guid,contract_positions_count
0,00026a57-3035-45cd-9a0b-598d5b233a6b,8
1,0004ff4b-67ff-42b6-a163-88e556affbeb,3
2,000ab495-ae5e-42c4-9643-94083ebb7ceb,21
3,000ccb0b-8346-4f62-bb32-04d284ba6fca,5
4,000d73ea-90c4-460f-8074-2d8eb70aa733,1
...,...,...
30698,fff5ad7e-5884-46f1-9f07-4150e8cc6258,1
30699,fff6f724-be51-4180-930f-41999fdd5497,1
30700,fff8c702-ba6a-48c2-90d4-67c2c857cfd1,1
30701,fffb9623-244a-4c27-b1c7-4518ab6412bd,1


In [16]:
dataset = pd.merge(
    dataset,
    contract_positions_count,
    how='left',
    on='contract_guid'
)

dataset


,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code,contracts_for_lot,notice_customer_inn,notice_placer_inn,notice_detached_org_inn,contract_positions_count
0,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Пара (2 шт.),4000.0,26.0,NaN,RUB,1,7017000450,7017000450,None,8
1,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Пара (2 шт.),5000.0,26.0,NaN,RUB,1,7017000450,7017000450,None,8
2,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Пара (2 шт.),1000.0,28.0,NaN,RUB,1,7017000450,7017000450,None,8
3,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Пара (2 шт.),100.0,54.0,NaN,RUB,1,7017000450,7017000450,None,8
4,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,Пара (2 шт.),8000.0,26.0,NaN,RUB,1,7017000450,7017000450,None,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72190,fff5ad7e-5884-46f1-9f07-4150e8cc6258,52801022793210003140001,0,0,0,0,None,None,2021-11-01 05:30:25,2801022793,...,Упаковка,63.0,5800.0,NaN,RUB,1,2801022793,2801022793,None,1
72191,fff6f724-be51-4180-930f-41999fdd5497,51435149216210002250001,0,0,0,0,None,None,2021-11-25 02:46:38,1435149216,...,None,NaN,617760.0,NaN,RUB,1,1435149216,7710357167,None,1
72192,fff8c702-ba6a-48c2-90d4-67c2c857cfd1,67725038124200003980001,0,0,0,0,None,None,2020-11-20 08:42:35,7725038124,...,Условная единица,1.0,NaN,NaN,None,1,7725038124,7725038124,None,1
72193,fffb9623-244a-4c27-b1c7-4518ab6412bd,51431004812210000980001,0,0,0,0,None,None,2021-07-07 09:05:27,1431004812,...,Упаковка,14.0,54600.0,NaN,RUB,1,1431004812,1431004812,None,1


Добавим в выборку информацию о поставщиках


In [17]:
lot_suppliers = pd.read_sql_query(
    f"""
    SELECT lot_guid, si2.inn as supplier_inn,
     si2.name as supplier_name, si2.ogrn as supplier_ogrn, si2.type as supplier_type
    FROM (
        SELECT lot_guid, MAX(supplier_guid) as supplier_guid
        FROM (
            SELECT c.lot_guid
            FROM contract c
            INNER JOIN (
                SELECT contract_reg_number, MAX(version) ver
                FROM contract
                WHERE {COVID_CONDITIONS}
                AND lot_guid is not null
                GROUP BY contract_reg_number
            ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
            GROUP BY c.lot_guid
        ) tmp1
        INNER JOIN protocol_lot pl ON tmp1.lot_guid = pl.guid
        LEFT JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
        LEFT JOIN application a on pla.id = a.protocol_lot_applications_id
        LEFT JOIN supplier_info si on a.supplier_guid = si.guid
        WHERE (contract_signed = 1 or
         (a.winner_indication = 'Победитель'and contract_signed is null))
         and (supplier_guid is not null)
        GROUP BY lot_guid
    ) sup
    LEFT JOIN supplier_info si2 ON si2.guid = sup.supplier_guid;
    """,
    conn
)

lot_suppliers

,lot_guid,supplier_inn,supplier_name,supplier_ogrn,supplier_type
0,001180aa-ec39-fe24-e63b-de59c6c8b320,7717729216,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВЫСО...",1127746547895,Юридическое лицо
1,002db7ef-9ca6-4b69-abce-04c3b9b7a06a,6452925517,Общество с ограниченной ответственностью «СОК»,1066450129382,Юридическое лицо
2,00305ade-c747-4999-b604-f4146cd4025d,551200523301,Бердник Елена Михайловна,None,Физическое лицо
3,0046cc75-3f65-47db-be53-2ad0ce7b202c,7453040371,Челябинское региональное отделение Общероссийс...,1207400002706,Юридическое лицо
4,005056bd-0450-1eea-aaf8-a92723b8381c,7817333828,"ООО ""Олимп Медикал""",1147847368217,Юридическое лицо
...,...,...,...,...,...
2762,ffaa3d53-69f7-48e7-a679-8cbc44eb7bbd,7725628067,"ООО ""БионаФарм""",1087746155111,Юридическое лицо
2763,ffb5bde5-3196-49ec-b1d8-9385a4bed2d0,7725628067,"ООО ""БионаФарм""",1087746155111,Юридическое лицо
2764,ffe29952-0bec-4deb-8422-0a118b0842e7,4003032047,Общество с ограниченной ответственностью «Торг...,1114025005720,Юридическое лицо
2765,ffe6f942-bfa1-4f78-bb1c-a8b0cf755881,None,ООО «СТАТУС-РЕГИОН»,None,Юридическое лицо


In [18]:
dataset = pd.merge(
    dataset,
    lot_suppliers,
    how='left',
    on='lot_guid'
)

dataset

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_currency_code,contracts_for_lot,notice_customer_inn,notice_placer_inn,notice_detached_org_inn,contract_positions_count,supplier_inn,supplier_name,supplier_ogrn,supplier_type
0,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,RUB,1,7017000450,7017000450,None,8,NaN,NaN,NaN,NaN
1,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,RUB,1,7017000450,7017000450,None,8,NaN,NaN,NaN,NaN
2,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,RUB,1,7017000450,7017000450,None,8,NaN,NaN,NaN,NaN
3,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,RUB,1,7017000450,7017000450,None,8,NaN,NaN,NaN,NaN
4,00026a57-3035-45cd-9a0b-598d5b233a6b,57017000450210000030001,0,0,0,0,None,None,2021-01-12 02:56:07,7017000450,...,RUB,1,7017000450,7017000450,None,8,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72190,fff5ad7e-5884-46f1-9f07-4150e8cc6258,52801022793210003140001,0,0,0,0,None,None,2021-11-01 05:30:25,2801022793,...,RUB,1,2801022793,2801022793,None,1,NaN,NaN,NaN,NaN
72191,fff6f724-be51-4180-930f-41999fdd5497,51435149216210002250001,0,0,0,0,None,None,2021-11-25 02:46:38,1435149216,...,RUB,1,1435149216,7710357167,None,1,NaN,NaN,NaN,NaN
72192,fff8c702-ba6a-48c2-90d4-67c2c857cfd1,67725038124200003980001,0,0,0,0,None,None,2020-11-20 08:42:35,7725038124,...,None,1,7725038124,7725038124,None,1,NaN,NaN,NaN,NaN
72193,fffb9623-244a-4c27-b1c7-4518ab6412bd,51431004812210000980001,0,0,0,0,None,None,2021-07-07 09:05:27,1431004812,...,RUB,1,1431004812,1431004812,None,1,NaN,NaN,NaN,NaN


In [19]:
from pathlib import Path
from typing import Union, Optional
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

def append_df_to_excel(
        filename: Union[str, Path],
        df: pd.DataFrame,
        sheet_name: str = 'Sheet1',
        max_col_width: int = 30,
        autofilter: bool = False,
        fmt_int: str = "#,##0",
        fmt_float: str = "#,##0.00",
        fmt_date: str = "yyyy-mm-dd",
        fmt_datetime: str = "yyyy-mm-dd hh:mm",
        truncate_sheet: bool = False,
        storage_options: Optional[dict] = None,
        **to_excel_kwargs
) -> None:
    def set_column_format(ws, column_letter, fmt):
        for cell in ws[column_letter]:
            cell.number_format = fmt
    filename = Path(filename)
    file_exists = filename.is_file()
    first_col = int(to_excel_kwargs.get("index", True)) + 1
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')
    if file_exists:
        wb = load_workbook(filename)
        sheets = {ws.title: ws for ws in wb.worksheets}

    with pd.ExcelWriter(
        filename.with_suffix(".xlsx"),
        engine="openpyxl",
        mode="a" if file_exists else "w",
        if_sheet_exists="new" if file_exists else None,
        date_format=fmt_date,
        datetime_format=fmt_datetime,
        storage_options=storage_options
    ) as writer:
        if file_exists:
            writer.book = wb
            if truncate_sheet and sheet_name in writer.book.sheetnames:
                idx = writer.book.sheetnames.index(sheet_name)
                writer.book.remove(writer.book.worksheets[idx])
                writer.book.create_sheet(sheet_name, idx)
            writer.sheets = sheets

        df.to_excel(writer, sheet_name=sheet_name, **to_excel_kwargs)
        worksheet = writer.sheets[sheet_name]

        if autofilter:
            worksheet.auto_filter.ref = worksheet.dimensions

        for xl_col_no, dtyp in enumerate(df.dtypes, first_col):
            col_no = xl_col_no - first_col
            width = max(df.iloc[:, col_no].astype(str).str.len().max(),
                        len(df.columns[col_no]) + 6)
            width = min(max_col_width, width)
            column_letter = get_column_letter(xl_col_no)
            worksheet.column_dimensions[column_letter].width = width
            if np.issubdtype(dtyp, np.integer):
                set_column_format(worksheet, column_letter, fmt_int)
            if np.issubdtype(dtyp, np.floating):
                set_column_format(worksheet, column_letter, fmt_float)

Выгрузим датасет в эксель

In [20]:
append_df_to_excel("output.xlsx", dataset, sheet_name="Main")

Найдём информацию о заявках, на интересующие нас лоты


In [25]:
lot_applications = pd.read_sql_query(
    f"""
    WITH covid_contract AS (
        SELECT *
        FROM contract c
        INNER JOIN (
            SELECT contract_reg_number, MAX(version) ver
            FROM contract
            WHERE {COVID_CONDITIONS}
            AND lot_guid is not null
            GROUP BY contract_reg_number
        ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    )
    SELECT l.lot_guid, pl.initial_sum as lot_initial_sum, pl.commodity_price as lot_commodity_price,
           pl.max_contract_price as lot_max_contract_price, pl.non_price as lot_non_price,
           pl.currency_code as lot_currency_code,
           a.application_date as application_date, a.not_dishonest as application_not_dishonest,
           a.provider as application_provider, a.supplier_guid as supplier_guid, si.inn as supplier_inn,
           si.name as supplier_name, si.ogrn as supplier_ogrn, si.type as supplier_type,
           a.price as application_price, a.currency_code as application_currency_code,
           a.accepted as application_accepted, a.application_rejection_reason_code as application_rejection_reason_code,
           a.winner_indication as application_winner_indication, a.contract_signed as application_contract_signed,
           a.additional_price as application_additional_price, a.rating as application_rating,
           a.assessment_result as application_assessment_result,
           a.commission_decision_place as application_commission_decision_place
    FROM (
        SELECT cc.lot_guid
        FROM covid_contract cc
        GROUP BY cc.lot_guid
    ) l
    INNER JOIN (
        SELECT cc2.lot_guid, MAX(pp2.create_date_time) as create_date_time FROM
        covid_contract cc2
        INNER JOIN protocol_lot pl2 ON cc2.lot_guid = pl2.guid
        LEFT JOIN protocol_lot_applications pla2 on pl2.guid = pla2.protocol_lot_guid
        LEFT JOIN purchase_protocol pp2 on pla2.purchase_protocol_guid = pp2.guid
        GROUP BY cc2.lot_guid
    ) tmp2 ON l.lot_guid = tmp2.lot_guid
    INNER JOIN protocol_lot pl ON l.lot_guid = pl.guid
    INNER JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
    INNER JOIN purchase_protocol pp on pla.purchase_protocol_guid = pp.guid and tmp2.create_date_time = pp.create_date_time
    INNER JOIN application a on pla.id = a.protocol_lot_applications_id
    LEFT JOIN supplier_info si on a.supplier_guid = si.guid
    ;
    """,
    conn
)

lot_applications

,lot_guid,lot_initial_sum,lot_commodity_price,lot_max_contract_price,lot_non_price,lot_currency_code,application_date,application_not_dishonest,application_provider,supplier_guid,...,application_price,application_currency_code,application_accepted,application_rejection_reason_code,application_winner_indication,application_contract_signed,application_additional_price,application_rating,application_assessment_result,application_commission_decision_place
0,00063eeb-9fc5-4bd1-b269-9ebf61448d5d,195264.00,None,NaN,NaN,RUB,2021-04-20 15:58:54,NaN,NaN,None,...,187600.00,RUB,Допущен,None,None,1.0,NaN,1.0,NaN,NaN
1,000b0a3c-7291-4077-836d-73b1ce173a73,826881.93,None,NaN,NaN,RUB,2020-07-13 06:11:07,NaN,NaN,None,...,797264.49,RUB,Допущен,None,None,0.0,NaN,NaN,NaN,2.0
2,000b0a3c-7291-4077-836d-73b1ce173a73,826881.93,None,NaN,NaN,RUB,2020-07-13 17:25:37,NaN,NaN,None,...,597995.04,RUB,Допущен,None,None,1.0,NaN,NaN,NaN,1.0
3,000f785b-cde8-42fc-bdd4-dd401bf48e52,NaN,None,574135.0,NaN,RUB,2020-03-17 13:45:04,NaN,NaN,None,...,574135.00,RUB,Допущен,None,None,1.0,NaN,1.0,NaN,NaN
4,000f785b-cde8-42fc-bdd4-dd401bf48e52,NaN,None,574135.0,NaN,RUB,2020-03-17 13:45:04,NaN,NaN,None,...,574135.00,RUB,Допущен,None,None,1.0,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44908,fff15ad5-3a2e-9349-9c82-fd90df956788,28200.00,None,NaN,NaN,RUB,2020-02-26 13:50:22,NaN,NaN,None,...,27000.00,RUB,Допущен,None,None,0.0,NaN,NaN,2.0,0.0
44909,fff3b4ba-20f9-451e-8ecd-d0c185e2cf56,5159837.73,None,NaN,NaN,RUB,2020-04-28 09:00:00,NaN,NaN,None,...,5159837.73,RUB,None,None,None,NaN,NaN,NaN,NaN,NaN
44910,fffdce4d-a139-4939-ada5-5bea3fb1d25e,499924.76,None,NaN,NaN,RUB,2021-01-25 16:08:08,NaN,NaN,None,...,497425.14,RUB,Допущен,None,None,0.0,NaN,NaN,NaN,2.0
44911,fffdce4d-a139-4939-ada5-5bea3fb1d25e,499924.76,None,NaN,NaN,RUB,2021-01-19 05:59:35,NaN,NaN,None,...,472428.90,RUB,Допущен,None,None,1.0,NaN,NaN,NaN,1.0


Выгрузим заявки в эксель

In [26]:
append_df_to_excel("output.xlsx", lot_applications, sheet_name="Applications")

Проанализируем исполнение договоров в нашей выборке:

In [23]:
failed_contracts = covid_contracts_performance_with_lot[
    covid_contracts_performance_with_lot["contract_is_complete"] == 0]
completed_contracts = covid_contracts_performance_with_lot[
    covid_contracts_performance_with_lot["contract_is_complete"] == 1]

contracts_count = covid_contracts_performance_with_lot.shape[0]
failed_contracts_count = failed_contracts.shape[0]
completed_contracts_count = completed_contracts.shape[0]

print(f'Всего договоров: {contracts_count}')
print(f'Всего завершенных договоров: {failed_contracts_count + completed_contracts_count}')
print(f'Всего расторженных договоров: {failed_contracts_count}')
print(f'Всего исполненных договоров: {completed_contracts_count}')
print(
    f'Процент расторженных договоров от завершенных: {failed_contracts_count / (failed_contracts_count + completed_contracts_count) * 100}%')
print(f'Процент расторженных договоров от всех: {failed_contracts_count / contracts_count * 100}%')

Всего договоров: 30703
Всего завершенных договоров: 21685
Всего расторженных договоров: 1090
Всего исполненных договоров: 20595
Процент расторженных договоров от завершенных: 5.026516024902006%
Процент расторженных договоров от всех: 3.550141679966127%


Проанализируем в каком количестве позиций присутствует цена за единицу

In [24]:
positions_count = covid_contracts_positions_with_lot.shape[0]
unit_price_positions_count = covid_contracts_positions_with_lot[covid_contracts_positions_with_lot["position_unit_price"].notnull()].shape[0]

print(f'Всего позиций: {positions_count}')
print(f'Всего позиций с указанной ценой: {unit_price_positions_count}')
print(f'Процент позиций с указанной ценой: {unit_price_positions_count / positions_count * 100}%')

Всего позиций: 72195
Всего позиций с указанной ценой: 42814
Процент позиций с указанной ценой: 59.303275850128124%
